In [5]:
import yaml
import os

In [9]:
os.getcwd()

'c:\\Users\\Admin\\OneDrive\\Desktop\\project\\Machine_learning_project'

In [8]:
os.chdir("c:\\Users\\Admin\\OneDrive\\Desktop\\project\\Machine_learning_project")

In [10]:
os.listdir(".")

['.dockerignore',
 '.git',
 '.github',
 '.gitignore',
 '.vscode',
 'app.py',
 'config',
 'Dockerfile',
 'housing',
 'housing_logs',
 'housing_predictor.egg-info',
 'LICENSE',
 'notebook',
 'README.md',
 'requirements.txt',
 'setup.py',
 'venv']

In [11]:
os.listdir("config")

['config.yaml']

In [13]:
config_file_path=os.path.join("config","config.yaml")
config_file_path

'config\\config.yaml'

In [15]:
os.path.exists(config_file_path)

True

In [16]:
config_info=None
with open(config_file_path,"rb") as yaml_file:
    config_info=yaml.safe_load(yaml_file)

In [17]:
config_info

{'training_pipeline_config': {'pipeline_name': 'housing',
  'artifact_dir': 'artifact'},
 'data_ingestion_config': {'dataset_download_url': 'https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz',
  'raw_data_dir': 'raw_data',
  'tgz_download_dir': 'tgz_data',
  'ingested_dir': 'ingested_data',
  'ingested_train_dir': 'train',
  'ingested_test_dir': 'test'},
 'data_validation_config': {'schema_dir': 'config',
  'schema_file_name': 'schema.yaml',
  'report_file_name': 'report.json',
  'report_page_file_name': 'report.html'},
 'data_transformation_config': {'add_bedroom_per_room': True,
  'transformed_dir': 'transformed_data',
  'transformed_train_dir': 'train',
  'transformed_test_dir': 'test',
  'preprocessing_dir': 'preprocessed',
  'preprocessed_object_file_name': 'preprocessed.pkl'},
 'model_trainer_config': {'trained_model_dir': 'trained_model',
  'model_file_name': 'model.pkl',
  'base_accuracy': 0.6,
  'model_config_dir': 'config',
  'model_confi